# Preliminaries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import sys
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

sys.path.append("/Users/paolo/Documents/methods/CMI_FS")
from feature_selection import forwardFeatureSelection

sys.path.append("/Users/paolo/Documents/methods/LinCFA")
from LinCFA import LinCFA

sys.path.append("/Users/paolo/Documents/Droughts/Paolo/regression_LinCFA")
from aux import standardize,unfold_dataset,compute_r2,prepare_target,prepare_features,aggregate_unfolded_data,FS_with_linearWrapper,compare_methods


In [2]:
def plot_cells(output,selected_colnames, xmin=9, xmax=11, ymin=44, ymax=45.5):
    x = []
    y = []
    colors = cm.rainbow(np.linspace(0,1,len(output)))
    fig, ax = plt.subplots(2)
    ax[0].set_xlim(xmin,xmax)
    ax[1].set_xlim(xmin,xmax)
    ax[0].set_ylim(ymin,ymax)
    ax[1].set_ylim(ymin,ymax)
    for i in range(len(output)): 
        #print(len(output[i]))
        x = []
        y = []
        
        for datum in output[i]:
            x.append(float(datum.split('_')[1]))
            y.append(float(datum.split('_')[2]))
        ax[0].scatter(x,y,color=colors[i])
    
    x = []
    y = []
    col = cm.rainbow(np.linspace(0,1,len(selected_colnames)))
    for i in range(len(selected_colnames)): 
        idx = int(selected_colnames[i].split('_')[-1])
        for datum in output[idx]:
            x.append(float(datum.split('_')[1]))
            y.append(float(datum.split('_')[2]))
        ax[1].scatter(x,y,color=col[i])
    

# Temperature

## LinCFA: Wrapper and CMI FS

In [10]:
path_target = '/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/csv_VHI/'
basins = ['Adda','Dora','Emiliani1','Emiliani2','Garda_Mincio','Lambro_Olona','Oglio_Iseo','Piemonte_Nord','Piemonte_Sud','Ticino']

path_features='/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/features/csv_allvalues/temporal_aggreg/'

for basin in basins:
    print('####################' + basin + '####################')
    target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01', path=path_target+basin+'.csv')
    actual_path = path_features+basin+'_aggreg.csv'
    df = pd.read_csv(actual_path)
    
    output,aggregate_trainVal,aggregate_test = aggregate_unfolded_data(actual_path,['cyclostationary_mean_tg', 
                                                                         'cyclostationary_mean_tg_1w',
                                                                         'cyclostationary_mean_tg_4w', 
                                                                         'cyclostationary_mean_tg_8w',
                                                                         'cyclostationary_mean_tg_12w', 
                                                                         'cyclostationary_mean_tg_16w',
                                                                         'cyclostationary_mean_tg_24w'
                                                                        ],
                                                                   target_df_trainVal, 
                                                                   max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01')
    
    selected_colnames = FS_with_linearWrapper(aggregate_trainVal, target_df_train, target_df_val, 50, 228)

    print('\nFull model and selected features with wrapper\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)
    print('\nFull model and best 5 selected features with wrapper\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])
    
    ### CMI FS
    res = {
        "delta" : [], 
        "numSelected" : [], 
        "selectedFeatures" : [] 
    }
    
    res['selectedFeatures'] = forwardFeatureSelection(10,np.array(aggregate_trainVal),np.array(target_df_trainVal.mean_std),res,10,1)
    
    selectedFeatures='selectedFeatures'
    print(f'\n{res[selectedFeatures]}\n')
    selected_colnames = aggregate_trainVal.columns[res['selectedFeatures']]

    print('\nFull model and selected features with CMI FS\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)
    print('\nFull model and best 5 selected features with CMI FS\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])

    
    print('\n\n\n')

####################Adda####################
target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.039373    0.00  2001     1 -2.546951
1    2001-01-13  0.380618    0.43  2001     2 -0.277191
2    2001-01-21  0.341985    0.38  2001     3 -0.534156
3    2001-01-29  0.322044    0.35  2001     5 -0.666789
4    2001-02-06  0.354954    0.40  2001     6 -0.447894
..          ...       ...     ...   ...   ...       ...
406  2009-11-27  0.382706    0.40  2009    48 -0.263306
407  2009-12-05  0.409921    0.46  2009    49 -0.082282
408  2009-12-13  0.472087    0.53  2009    50  0.331204
409  2009-12-21  0.324728    0.00  2009    52 -0.648940
410  2009-12-29  0.086512    0.00  2009    53 -2.233412

[411 rows x 6 columns]
 target shapes: ((411, 6), (228, 6), (639, 6), (228, 6))
Number of features: 92

Number of aggregated features: 41

Number of features: 92

Number of aggregated features: 36

Number of features: 92

Number of aggregated features: 34

Number of

----- MI Scores -----
[(21, 0.1380768961183111), (26, 0.13316272799876147), (52, 0.1311367660157456), (17, 0.12580027229801832), (51, 0.12412903719593178), (59, 0.12283641167559335), (20, 0.1221831296200904), (69, 0.12169626223125608), (56, 0.12144649716454456), (42, 0.12080119282265736), (43, 0.11918475260343826), (3, 0.1187066439225602), (65, 0.11794701080259946), (19, 0.11581685950856832), (53, 0.11568527876685653), (13, 0.11562685871926665), (55, 0.1148414002210356), (9, 0.11468357289394793), (11, 0.11463699077576713), (68, 0.1139207550391548), (34, 0.11286256704906758), (2, 0.11065987920447934), (12, 0.11055746898233354), (39, 0.11045881633015249), (8, 0.11044903362598607), (44, 0.10957762389085016), (66, 0.10914206632698413), (58, 0.1088993837968551), (31, 0.10882985460336782), (1, 0.10810649608547497), (60, 0.10700871250550935), (33, 0.1069164602831172), (61, 0.10648272158241143), (57, 0.10608404790220755), (63, 0.10478136765678527), (70, 0.10455074622644751), (4, 0.104448579230

Number of features: 44

Number of aggregated features: 7

Number of features: 44

Number of aggregated features: 16

Number of features: 44

Number of aggregated features: 13

Number of features: 44

Number of aggregated features: 15

Number of features: 44

Number of aggregated features: 22

Number of features: 44

Number of aggregated features: 21

Number of features: 44

Number of aggregated features: 23

actual training score: 0.028874764571624745
actual validation score: 0.07896390569144529, number of remaining columns: 115

actual training score: 0.03620885626339032
actual validation score: 0.17762970908654074, number of remaining columns: 114

actual training score: 0.03812083383825404
actual validation score: 0.19309299432014992, number of remaining columns: 113

actual training score: 0.04206602706997853
actual validation score: 0.20587956068055424, number of remaining columns: 112

actual training score: 0.0444552322686409
actual validation score: 0.2082918678187522, number o

CMI: 0.014414293705003495
CMI: 0.012095740709347588
CMI: 0.010492151602611924
CMI: 0.025421283498248987
CMI: 0.00856669151760732
CMI: 0.018259049365115756
CMI: 0.00403211923419966
CMI: 0.007785968749108474
CMI: 0.011293333453075746
CMI: 0.0017525025782159642
CMI: 0.019941983410870348
CMI: 0.006576666828004535
CMI: 0.032981739417747485
CMI: 0.01912194118134479
CMI: 0.00857434332478707
CMI: 0.006532454428670287
CMI: 0.009011399288896396
CMI: 0.01720991230393179
CMI: 0.04759772888773564
CMI: 0.01742315194745167
CMI: 0.017753554582296507
CMI: 0.022722951359491247
CMI: 0.01766711120167694
CMI: 0.017244797967931977
CMI: 0.01643287860046419
CMI: 0.013187940388864755
CMI: 0.005138970889477443
CMI: 0.018325111220345386
CMI: 0.012503746688990361
CMI: 0.007990424664348836
CMI: 0.016435822180935103
CMI: 0.022491530814550492
CMI: 0.00031637078151025566
CMI: 0.008444979055370797
CMI: 0.011644579727391982
CMI: 0.016449914742544397
CMI: 0.0035111138953345805
CMI: 0.009081195267226574
CMI: 0.0153105827

actual training score: 0.26611709380585213
actual validation score: 0.5405418731673088, number of remaining columns: 461

actual training score: 0.26632700060569425
actual validation score: 0.5418899674761766, number of remaining columns: 460

actual training score: 0.26676336057462713
actual validation score: 0.5426592590680732, number of remaining columns: 459

actual training score: 0.2667985526172787
actual validation score: 0.5428188713829172, number of remaining columns: 458

actual training score: 0.2667987697706674
actual validation score: 0.5428578190977082, number of remaining columns: 457

actual training score: 0.26680153793229977
actual validation score: 0.5428448786211135, number of remaining columns: 456

actual training score: 0.2668070462158211
actual validation score: 0.5428126245573592, number of remaining columns: 455

actual training score: 0.26680833115320957
actual validation score: 0.542675862692775, number of remaining columns: 454

actual training score: 0.267

...

CMI: 0.017887197889988382
CMI: 0.011265823824092885
CMI: 0.004545928279771461
CMI: 0.01247439439256856
CMI: 0.015847319406609578
CMI: 0.01962554807879152
CMI: 0.005824610352182916
CMI: 0.014692529797457915
CMI: 0.02633143073297095
CMI: 0.005281630580774077
CMI: 0.005664540803678106
CMI: 0.003724568469038514
CMI: 0.005430972678076365
CMI: 0.011361326487959067
CMI: 0.006179838074805516
CMI: 0.005487859965940492
CMI: 0.021963245202343606
CMI: 0.0010670534473727622
CMI: 0.007058547135461335
CMI: 0.00794647373301094
CMI: 0.0071026256576673386
CMI: 0.009312863642794877
CMI: 0.0016535983358294815
CMI: 0.013549584299331063
CMI: 0.007110941920407954
CMI: 0.003165207950509813
CMI: 0.0006925238407042844
CMI: 0.009372515669058196
CMI: 0.011326929049544798
CMI: 0.008318496078496301
CMI: 0.004615461048314132
CMI: 0.00647289766731074
CMI: 0.002358110914159134
CMI: 0.004864937349980683
CMI: 0.0022291354104308075
CMI: 0.00016177051084949323
CMI: 0.007322791415866531
CMI: 0.006231784248554778
CMI: 0.012

actual training score: 0.1465756804504723
actual validation score: 0.3699115884988923, number of remaining columns: 182

actual training score: 0.14684096165843363
actual validation score: 0.37286961046113953, number of remaining columns: 181

actual training score: 0.14727548414999736
actual validation score: 0.37361060164972315, number of remaining columns: 180

actual training score: 0.1562412176837652
actual validation score: 0.3775097057176171, number of remaining columns: 179

actual training score: 0.15822705671091197
actual validation score: 0.3816970953329156, number of remaining columns: 178

actual training score: 0.15997330838213886
actual validation score: 0.38462623091561443, number of remaining columns: 177

actual training score: 0.16014363667126297
actual validation score: 0.38794102933137675, number of remaining columns: 176

actual training score: 0.16129023154458866
actual validation score: 0.3910446324096032, number of remaining columns: 175

actual training score:

CMI: 0.005890700997267101
CMI: 0.0012998864986875774
CMI: 0.005857092131510838
CMI: 0.008571791467606393
CMI: 0.0054666669526303585
CMI: 0.0030912880291352335
CMI: 0.0016882985474157147
CMI: 0.008228919138834667
CMI: 2.3722092284475527e-05
CMI: 0.016051445193029645
CMI: 7.639093782367545e-05
CMI: 0.0076923090429759505
CMI: 0.019342789499840113
CMI: 0.004730331585492867
CMI: 0.007604809296491377
CMI: 0.0037028053386003734
CMI: 0.010640426330208716
CMI: 0.010562262225923658
CMI: 0.007036182669249602
CMI: 0.007748618152306722
CMI: 0.00828855128754781
CMI: 0.01031262110332265
CMI: 0.001762121531325822
CMI: 0.000531786163311454
CMI: 0.0010726571632984072
CMI: 0.015052821407895892
CMI: 0.010354628447980158
CMI: 0.01751477257865945
CMI: 0.02161999236997031
CMI: 0.01564819732627963
CMI: 0.021684345440327574
CMI: 0.013056578789540543
Highest CMI score: 0.021684345440327574
Adding original feature: 188
CMI: 0.00010408690539498444
CMI: 0.0014776913510133882
CMI: 0.0009715984678509704
CMI: 0.00124

actual training score: 0.11220126204721859
actual validation score: 0.4008477401863494, number of remaining columns: 182

actual training score: 0.11269463306412142
actual validation score: 0.4051229040260812, number of remaining columns: 181

actual training score: 0.11353287971453996
actual validation score: 0.40788542324768007, number of remaining columns: 180

actual training score: 0.11453660675571964
actual validation score: 0.41503450558312205, number of remaining columns: 179

actual training score: 0.11629764249441299
actual validation score: 0.41904829420395084, number of remaining columns: 178

actual training score: 0.11889271220603903
actual validation score: 0.4325742935358149, number of remaining columns: 177

actual training score: 0.11907553906105228
actual validation score: 0.4370277494702771, number of remaining columns: 176

actual training score: 0.1191360932076998
actual validation score: 0.4383437773926714, number of remaining columns: 175

actual training score:

CMI: 0.020832839454174135
CMI: 0.0007804825639846774
CMI: 0.0008553702560377269
CMI: 0.016678316258438602
CMI: 0.012444770705334174
CMI: 0.014988825041601761
CMI: 0.0013578905781041067
CMI: 6.117296787026827e-05
CMI: 0.00572605660142847
CMI: 0.0038909146022563512
CMI: 0.002904519789224652
CMI: 0.006982216614379505
CMI: 0.004012254952768329
CMI: 0.007329937508767093
CMI: 0.003881468991700582
CMI: 0.0014088314287769577
CMI: 0.007717242652303946
CMI: 0.0173172170680381
CMI: 0.018340462701497334
CMI: 0.006454119323864083
CMI: 0.03356237163547959
CMI: 0.0018816008330788253
CMI: 0.010554474734789265
CMI: 0.022953150965986907
CMI: 0.0025942949569773277
CMI: 0.017374807372951356
CMI: 0.018520015436354925
CMI: 0.01558193391161053
CMI: 0.010182749325042068
CMI: 8.04988115028632e-05
CMI: 0.005684595249937613
CMI: 0.004386194844045094
CMI: 0.006390876127572054
CMI: 0.008888359519881067
CMI: 5.277677932809255e-05
CMI: 0.004671525348574207
CMI: 0.009406308295980076
CMI: 0.005592986412337159
CMI: 0.0

# precipitation

## LinCFA: Wrapper and CMI FS

In [11]:
path_target = '/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/csv_VHI/'
basins = ['Adda','Dora','Emiliani1','Emiliani2','Garda_Mincio','Lambro_Olona','Oglio_Iseo','Piemonte_Nord','Piemonte_Sud','Ticino']

path_features='/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/features/csv_allvalues/temporal_aggreg/'

for basin in basins:
    print('####################' + basin + '####################')
    target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01', path=path_target+basin+'.csv')
    actual_path = path_features+basin+'_aggreg.csv'
    df = pd.read_csv(actual_path)
    
    output,aggregate_trainVal,aggregate_test = aggregate_unfolded_data(actual_path,['cyclostationary_mean_rr', 
                                                                         'cyclostationary_mean_rr_1w',
                                                                         'cyclostationary_mean_rr_4w', 
                                                                         'cyclostationary_mean_rr_8w',
                                                                         'cyclostationary_mean_rr_12w', 
                                                                         'cyclostationary_mean_rr_16w',
                                                                         'cyclostationary_mean_rr_24w'
                                                                        ],
                                                                   target_df_trainVal, 
                                                                   max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01')
    
    selected_colnames = FS_with_linearWrapper(aggregate_trainVal, target_df_train, target_df_val, 50, 228)

    print('\nFull model and selected features with wrapper\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)
    print('\nFull model and best 5 selected features with wrapper\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])
    
    ### CMI FS
    res = {
        "delta" : [], 
        "numSelected" : [], 
        "selectedFeatures" : [] 
    }
    
    res['selectedFeatures'] = forwardFeatureSelection(10,np.array(aggregate_trainVal),np.array(target_df_trainVal.mean_std),res,10,1)
    
    selectedFeatures='selectedFeatures'
    print(f'\n{res[selectedFeatures]}\n')
    selected_colnames = aggregate_trainVal.columns[res['selectedFeatures']]

    print('\nFull model and selected features with CMI FS\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)
    print('\nFull model and best 5 selected features with CMI FS\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])

    
    print('\n\n\n')

####################Adda####################
target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.039373    0.00  2001     1 -2.546951
1    2001-01-13  0.380618    0.43  2001     2 -0.277191
2    2001-01-21  0.341985    0.38  2001     3 -0.534156
3    2001-01-29  0.322044    0.35  2001     5 -0.666789
4    2001-02-06  0.354954    0.40  2001     6 -0.447894
..          ...       ...     ...   ...   ...       ...
406  2009-11-27  0.382706    0.40  2009    48 -0.263306
407  2009-12-05  0.409921    0.46  2009    49 -0.082282
408  2009-12-13  0.472087    0.53  2009    50  0.331204
409  2009-12-21  0.324728    0.00  2009    52 -0.648940
410  2009-12-29  0.086512    0.00  2009    53 -2.233412

[411 rows x 6 columns]
 target shapes: ((411, 6), (228, 6), (639, 6), (228, 6))
Number of features: 92

Number of aggregated features: 12

Number of features: 92

Number of aggregated features: 25

Number of features: 92

Number of aggregated features: 24

Number of

----- MI Scores -----
[(17, 0.08538288551780161), (26, 0.0827433257719066), (20, 0.0720018413805457), (14, 0.06977547569841301), (59, 0.0684758112694174), (16, 0.06791149304307495), (61, 0.06643822261845127), (25, 0.06546672215338904), (36, 0.06458783466411099), (45, 0.06201655529889099), (24, 0.060198517223907407), (49, 0.05919012181002004), (33, 0.0583357288761217), (38, 0.05820828380345304), (44, 0.05695900884176582), (42, 0.05558045359113545), (40, 0.054626698140009186), (12, 0.054074676873761836), (23, 0.054052397021323605), (137, 0.053800864642272406), (48, 0.05319276783685461), (142, 0.052979691061357206), (62, 0.05281860721884411), (47, 0.05238221210285017), (32, 0.05210627006643853), (43, 0.052056741071851), (63, 0.052010377086860377), (57, 0.05190749031926515), (179, 0.05143480244707808), (52, 0.05101251229387421), (160, 0.05019309619350385), (35, 0.05000631678807994), (31, 0.04960437727794816), (30, 0.04956122075339866), (50, 0.04945934102621864), (58, 0.04942334336961483), 

CMI: 0.0002517308300388488
CMI: 0.0006812006168331608
Highest CMI score: 0.0028959855034224324
Adding original feature: 122
CMI: 0.00382524882574925
CMI: 0.0008181821238049669
CMI: 0.00736256261008994
CMI: 0.0005746582796570532
CMI: 0.002292646862081499
CMI: 0.0023271623259562557
CMI: 0.0013613218911813496
CMI: 0.001953658299736649
CMI: 0.0011285009287138603
CMI: 0.00015524042182224473
CMI: 0.0012973923460568026
CMI: 0.0035879512769331767
CMI: 0.0003208463613142143
CMI: 0.0008723834001287678
CMI: 0.0020026326017573703
CMI: 0.0033581475052071874
CMI: 0.0039728969415253335
CMI: 0.0010044384190465716
Highest CMI score: 0.00736256261008994
Adding original feature: 71
CMI: 0.004697941998513394
CMI: 0.0013734547799674202
Highest CMI score: 0.004697941998513394
Adding original feature: 93
Highest CMI score: -0.0009368253073795685

[17, 145, 90, 12, 137, 122, 71, 93]


Full model and selected features with CMI FS

Full aggregate regression train score: 0.4697654674037829, test score: -2.671030

actual training score: 0.28860894673246695
actual validation score: -1.5748373716993171, number of remaining columns: 6

actual training score: 0.2909758318212329
actual validation score: -1.6399287131775715, number of remaining columns: 5



selected columns: ['cyclostationary_mean_rr_16w_4', 'cyclostationary_mean_rr_1w_2', 'cyclostationary_mean_rr_8w_8', 'cyclostationary_mean_rr_8w_5', 'cyclostationary_mean_rr_24w_3', 'cyclostationary_mean_rr_24w_4', 'cyclostationary_mean_rr_4w_0', 'cyclostationary_mean_rr_4w_3'], 

validation score: 0.08289163412774503, 

number of selected features: 8

Full model and selected features with wrapper

Full aggregate regression train score: 0.18762409497093602, test score: -3.695638853500318
Aggregate regression train score with FS: 0.09133255583490785, test score: -1.3921811600603382

Full model and best 5 selected features with wrapper

Full aggregate regression train score: 0.18762409497093602, test score: -3.695638853500318
Aggregate regression tra

actual training score: 0.2925820619203028
actual validation score: 0.4681012556680272, number of remaining columns: 271

actual training score: 0.2938401666953593
actual validation score: 0.47318555046809696, number of remaining columns: 270

actual training score: 0.3016961166147859
actual validation score: 0.4800577421224298, number of remaining columns: 269

actual training score: 0.3026669804078148
actual validation score: 0.485577759277383, number of remaining columns: 268

actual training score: 0.3040689514361701
actual validation score: 0.48983580969830953, number of remaining columns: 267

actual training score: 0.3067976751350209
actual validation score: 0.4937574750042517, number of remaining columns: 266

actual training score: 0.31084570346009177
actual validation score: 0.5008918697867035, number of remaining columns: 265

actual training score: 0.31182726218032253
actual validation score: 0.5063681992060103, number of remaining columns: 264

actual training score: 0.3130

...

CMI: 0.004192078302900082
CMI: 0.01014912374954191
CMI: 0.0014329105317934243
CMI: 0.005416347401821481
CMI: 0.0022113795167911787
CMI: 0.00010056415000408991
CMI: 0.003398066435076147
Highest CMI score: 0.01768163374283016
Adding original feature: 66
CMI: 0.0009537160959190438
CMI: 0.004496843730793565
CMI: 3.66824454855752e-05
CMI: 0.002246420497756141
CMI: 0.007273293217927357
CMI: 0.006518737785427858
CMI: 0.005990385201877002
CMI: 0.00025351214769628316
Highest CMI score: 0.007273293217927357
Adding original feature: 61
CMI: 0.0025844756010639514
CMI: 0.0016640399212098367
CMI: 0.00032561321859764236
CMI: 0.001075024596584595
CMI: 0.0016437994818324086
CMI: 0.00013775136735341698
CMI: 0.0012022554843942046
CMI: 0.0016282710830458902
CMI: 0.006253639050123022
CMI: 0.004216918816898563
CMI: 0.0004283379607017135
CMI: 0.0009601475987419117
Highest CMI score: 0.006253639050123022
Adding original feature: 69
CMI: 0.003824937456313765
CMI: 0.004891346729927415
CMI: 0.003543233170969412


actual training score: 0.22380395838704314
actual validation score: 0.35076936412326165, number of remaining columns: 137

actual training score: 0.22381179033456267
actual validation score: 0.3506668286572858, number of remaining columns: 136

actual training score: 0.22382069784104286
actual validation score: 0.3505566130993769, number of remaining columns: 135

actual training score: 0.22449097748384395
actual validation score: 0.34823460796576, number of remaining columns: 134

actual training score: 0.22490260806585627
actual validation score: 0.3450657896016157, number of remaining columns: 133

actual training score: 0.22526230041278306
actual validation score: 0.3421743535841545, number of remaining columns: 132

actual training score: 0.22636009694959258
actual validation score: 0.3398903305604153, number of remaining columns: 131

actual training score: 0.2273118342915872
actual validation score: 0.3393550884456009, number of remaining columns: 130

actual training score: 0.2

CMI: 0.011942948141803314
CMI: 0.013544170224600763
CMI: 0.007337298237790446
CMI: 0.0040961994063468815
CMI: 0.0008316478387364323
CMI: 0.005427225792787885
CMI: 0.014525211335239332
CMI: 0.006653748002054621
CMI: 0.012855043810014469
CMI: 0.028571634476215593
CMI: 0.03361382105042267
CMI: 0.010965120052817592
CMI: 0.02187816254127524
CMI: 0.010956288026404429
CMI: 0.006418685965540211
CMI: 0.0388969435228967
CMI: 0.014562515771260678
CMI: 0.005007440986781034
CMI: 0.009654806906717953
CMI: 0.040558685570992806
CMI: 0.013709779583900614
CMI: 0.010308166591225812
CMI: 0.01037245349044695
CMI: 0.038141380466481176
CMI: 0.03237388563209047
CMI: 0.014527414433627081
CMI: 0.016139050746200687
CMI: 0.0018121504185744652
CMI: 0.0034669425621656202
CMI: 0.017248348078837752
CMI: 0.006656263017428721
CMI: 0.0015379920654844254
CMI: 0.007165777744242485
CMI: 0.006013660758722186
CMI: 0.005183902232581
CMI: 0.000318528295701892
CMI: 0.005407499410391078
CMI: 0.01204384756469322
CMI: 0.0170840282

actual training score: 0.07517489953897394
actual validation score: 0.11780715927153584, number of remaining columns: 95

actual training score: 0.08921400127662682
actual validation score: 0.1290441244734828, number of remaining columns: 94

actual training score: 0.09083034630822229
actual validation score: 0.1347318617301111, number of remaining columns: 93

actual training score: 0.10830724517698898
actual validation score: 0.14105005002370452, number of remaining columns: 92

actual training score: 0.11305589594388588
actual validation score: 0.16517067578815947, number of remaining columns: 91

actual training score: 0.11432710007021052
actual validation score: 0.16908249903663952, number of remaining columns: 90

actual training score: 0.1143802104044952
actual validation score: 0.17170658690242668, number of remaining columns: 89

actual training score: 0.11929439098905115
actual validation score: 0.17381864892745824, number of remaining columns: 88

actual training score: 0.11

CMI: 0.004700538352927275
CMI: 0.01019315046404537
CMI: 0.019711024217264228
Highest CMI score: 0.019711024217264228
Adding original feature: 23
CMI: 0.007069570611589318
Highest CMI score: 0.007069570611589318
Adding original feature: 22
CMI: 0.006044068732098298
CMI: 0.006074071905212766
CMI: 0.007566300795637082
Highest CMI score: 0.007566300795637082
Adding original feature: 13
CMI: 0.0038615923188084073
CMI: 0.001756268216016807
CMI: 0.0036053343291626017
CMI: 0.0032847722599722434
CMI: 0.0007384645783308219
CMI: 0.0011049125285688133
CMI: 0.00346567767734543
Highest CMI score: 0.0038615923188084073
Adding original feature: 17
CMI: 0.003879485674294747
CMI: 0.000681844355163122
CMI: 0.0004084723449280514
CMI: 0.0035990761169342744
CMI: 0.00022842299344534578
Highest CMI score: 0.003879485674294747
Adding original feature: 8
CMI: 0.0004482317291089333
CMI: 0.0030582476308445017
CMI: 0.009918802280020164
CMI: 0.006244162327449648
CMI: 0.002912284586829572
Highest CMI score: 0.009918

# Both

## LinCFA: Wrapper and CMI FS

In [12]:
path_target = '/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/csv_VHI/'
basins = ['Adda','Dora','Emiliani1','Emiliani2','Garda_Mincio','Lambro_Olona','Oglio_Iseo','Piemonte_Nord','Piemonte_Sud','Ticino']

path_features='/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/features/csv_allvalues/temporal_aggreg/'

for basin in basins:
    print('####################' + basin + '####################')
    target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01', path=path_target+basin+'.csv')
    actual_path = path_features+basin+'_aggreg.csv'
    df = pd.read_csv(actual_path)
    
    output,aggregate_trainVal,aggregate_test = aggregate_unfolded_data(actual_path,['cyclostationary_mean_rr', 
                                                                         'cyclostationary_mean_rr_1w',
                                                                         'cyclostationary_mean_rr_4w', 
                                                                         'cyclostationary_mean_rr_8w',
                                                                         'cyclostationary_mean_rr_12w', 
                                                                         'cyclostationary_mean_rr_16w',
                                                                         'cyclostationary_mean_rr_24w',
                                                                         'cyclostationary_mean_tg', 
                                                                         'cyclostationary_mean_tg_1w',
                                                                         'cyclostationary_mean_tg_4w', 
                                                                         'cyclostationary_mean_tg_8w',
                                                                         'cyclostationary_mean_tg_12w', 
                                                                         'cyclostationary_mean_tg_16w',
                                                                         'cyclostationary_mean_tg_24w'
                                                                        ],
                                                                   target_df_trainVal, 
                                                                   max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01')
    
    selected_colnames = FS_with_linearWrapper(aggregate_trainVal, target_df_train, target_df_val, 50, 228)

    print('\nFull model and selected features with wrapper\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)
    print('\nFull model and best 5 selected features with wrapper\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])
    
    ### CMI FS
    res = {
        "delta" : [], 
        "numSelected" : [], 
        "selectedFeatures" : [] 
    }
    
    res['selectedFeatures'] = forwardFeatureSelection(10,np.array(aggregate_trainVal),np.array(target_df_trainVal.mean_std),res,10,1)
    
    selectedFeatures='selectedFeatures'
    print(f'\n{res[selectedFeatures]}\n')
    selected_colnames = aggregate_trainVal.columns[res['selectedFeatures']]

    print('\nFull model and selected features with CMI FS\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)
    print('\nFull model and best 5 selected features with CMI FS\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])

    
    print('\n\n\n')

####################Adda####################
target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.039373    0.00  2001     1 -2.546951
1    2001-01-13  0.380618    0.43  2001     2 -0.277191
2    2001-01-21  0.341985    0.38  2001     3 -0.534156
3    2001-01-29  0.322044    0.35  2001     5 -0.666789
4    2001-02-06  0.354954    0.40  2001     6 -0.447894
..          ...       ...     ...   ...   ...       ...
406  2009-11-27  0.382706    0.40  2009    48 -0.263306
407  2009-12-05  0.409921    0.46  2009    49 -0.082282
408  2009-12-13  0.472087    0.53  2009    50  0.331204
409  2009-12-21  0.324728    0.00  2009    52 -0.648940
410  2009-12-29  0.086512    0.00  2009    53 -2.233412

[411 rows x 6 columns]
 target shapes: ((411, 6), (228, 6), (639, 6), (228, 6))
Number of features: 92

Number of aggregated features: 12

Number of features: 92

Number of aggregated features: 25

Number of features: 92

Number of aggregated features: 24

Number of

----- MI Scores -----
[(209, 0.1380768961183111), (214, 0.13316272799876147), (240, 0.1311367660157456), (205, 0.12580027229801832), (239, 0.12412903719593178), (247, 0.12283641167559335), (208, 0.1221831296200904), (257, 0.12169626223125608), (244, 0.12144649716454456), (230, 0.12080119282265736), (231, 0.11918475260343826), (191, 0.1187066439225602), (253, 0.11794701080259946), (207, 0.11581685950856832), (241, 0.11568527876685653), (201, 0.11562685871926665), (243, 0.1148414002210356), (197, 0.11468357289394793), (199, 0.11463699077576713), (256, 0.1139207550391548), (222, 0.11286256704906758), (190, 0.11065987920447934), (200, 0.11055746898233354), (227, 0.11045881633015249), (196, 0.11044903362598607), (232, 0.10957762389085016), (254, 0.10914206632698413), (246, 0.1088993837968551), (219, 0.10882985460336782), (189, 0.10810649608547497), (248, 0.10700871250550935), (221, 0.1069164602831172), (249, 0.10648272158241143), (245, 0.10608404790220755), (251, 0.10478136765678527), (258,

CMI: 6.431709006449182e-05
CMI: 0.006918291858025255
CMI: 0.0018783026711822748
CMI: 0.008795524640894586
CMI: 0.00029349090883118434
CMI: 0.008971321189420567
CMI: 0.0030233517320346903
CMI: 0.006751047790119452
CMI: 0.0022973179598191895
CMI: 0.018305584060933078
CMI: 0.009772892105004927
CMI: 0.015063484121156323
CMI: 0.016529988520980282
CMI: 0.018313282167880862
CMI: 0.02005251486637291
CMI: 0.017278934276486746
CMI: 0.027836240331982426
CMI: 0.016984947200688383
CMI: 0.027600554461346793
CMI: 0.016582152626380908
CMI: 0.006161299601752407
CMI: 0.014251557316181052
CMI: 0.018048207650567943
CMI: 0.0060650265072778875
CMI: 0.017021787156141138
CMI: 0.009318858648866779
CMI: 0.0074906281521140705
CMI: 0.015503129589361286
CMI: 0.0027383643332715424
CMI: 0.014343235927807924
CMI: 0.006438697531493093
CMI: 0.008224430032876262
CMI: 0.02422556284942165
CMI: 0.017197268517177983
CMI: 0.011271802813947962
CMI: 0.013263600221030819
CMI: 0.000508241787399305
CMI: 0.014692195031683203
CMI: 

actual training score: 0.08192469393837287
actual validation score: 0.2289403498797138, number of remaining columns: 165

actual training score: 0.08207164633817632
actual validation score: 0.23270021067676383, number of remaining columns: 164

actual training score: 0.09873237006386404
actual validation score: 0.24045115158353392, number of remaining columns: 163

actual training score: 0.10229239723973271
actual validation score: 0.24704769276042715, number of remaining columns: 162

actual training score: 0.10313181923883319
actual validation score: 0.2529858114407155, number of remaining columns: 161

actual training score: 0.10381322235003387
actual validation score: 0.2608819374070923, number of remaining columns: 160

actual training score: 0.10475057240019037
actual validation score: 0.26962790468208997, number of remaining columns: 159

actual training score: 0.10653077972493419
actual validation score: 0.27690274169744555, number of remaining columns: 158

actual training sco

CMI: 0.005271628389310251
CMI: 0.025015655898955103
CMI: 0.012076342652769342
CMI: 0.010412122292021625
CMI: 0.0015716052327979652
CMI: 0.03677621469327799
CMI: 0.04004349178380662
CMI: 0.023748424346089897
CMI: 0.025616215349120897
CMI: 0.005426823892180674
CMI: 1.323794218897889e-05
CMI: 0.030714476878498653
CMI: 0.01775904142820567
CMI: 0.019292753802323037
CMI: 0.025376327555618416
CMI: 0.028598715371033284
CMI: 0.02400928895300089
CMI: 0.025198158791159858
CMI: 0.03340761127480302
CMI: 0.030546933225225448
CMI: 0.02521487025468097
CMI: 0.037548096102327205
CMI: 0.03859058277268382
CMI: 0.02639336872111528
CMI: 0.03748086090702976
CMI: 0.0192571540904979
CMI: 0.0324987605875558
CMI: 0.036655832847625885
CMI: 0.03911474410431809
CMI: 0.026514461410749535
CMI: 0.04170005425826301
CMI: 0.022897387399431007
CMI: 0.0319030370410211
CMI: 0.012077025248145162
CMI: 0.03235045644830384
CMI: 0.015492264907419423
CMI: 0.01630879126824565
CMI: 0.0135077364180059
CMI: 0.015177305370921351
CMI: 

...

CMI: 0.004675931563381655
CMI: 0.01087606699266723
CMI: 0.004144413753419757
CMI: 0.008841394285165768
CMI: 0.010367279880109145
CMI: 0.0029110186918617775
CMI: 0.000368707813223873
CMI: 0.0012330994479825136
CMI: 0.00434196232136852
CMI: 0.0009475712840774503
CMI: 0.001334197879219956
CMI: 0.03785444093243355
CMI: 0.014607502028582395
CMI: 0.0036607612089972763
CMI: 0.010381229410015028
CMI: 0.011942948141803314
CMI: 0.013544170224600763
CMI: 0.007337298237790446
CMI: 0.0040961994063468815
CMI: 0.0008316478387364323
CMI: 0.005427225792787885
CMI: 0.014525211335239332
CMI: 0.006653748002054621
CMI: 0.012855043810014469
CMI: 0.028571634476215593
CMI: 0.03361382105042267
CMI: 0.010965120052817592
CMI: 0.02187816254127524
CMI: 0.010956288026404429
CMI: 0.006418685965540211
CMI: 0.0388969435228967
CMI: 0.014562515771260678
CMI: 0.005007440986781034
CMI: 0.009654806906717953
CMI: 0.040558685570992806
CMI: 0.013709779583900614
CMI: 0.010308166591225812
CMI: 0.01037245349044695
CMI: 0.0381413

CMI: 0.007451719183845962
CMI: 0.0166881719544541
CMI: 0.0158722442426934
CMI: 0.01202794144526273
CMI: 0.0026971477156858825
CMI: 0.0008253428429718657
CMI: 0.007366481177476375
CMI: 0.0183153226017784
CMI: 0.00939896801035589
CMI: 0.022842819132795678
CMI: 0.026147146227632312
CMI: 0.009154109012632833
CMI: 0.0054860901582143295
CMI: 0.009896055367499518
CMI: 0.013638254889081741
CMI: 0.0034075789243152854
CMI: 0.005945401390870325
CMI: 0.0049546426489153195
CMI: 0.015539011567458322
CMI: 0.014627905006804165
CMI: 0.008028124939224834
CMI: 0.0018934690866494464
CMI: 0.0006834987199290377
CMI: 0.001775162885135989
CMI: 8.523529536297314e-05
CMI: 0.00991403560242754
CMI: 0.0015474675595561205
CMI: 0.008562394225577002
CMI: 0.006589906370970788
CMI: 0.007158927934176579
CMI: 0.0005863622718838835
CMI: 0.005903457024756142
CMI: 0.0010503094429970272
CMI: 0.002448905585891503
CMI: 0.0010135902082374082
CMI: 0.022127368202026598
CMI: 0.02381311573189515
CMI: 0.016887258976924113
CMI: 0.009

CMI: 0.01482431178817284
CMI: 0.01587003264745812
CMI: 0.01595520175204454
CMI: 0.013108202677460629
CMI: 0.01216211812038312
CMI: 0.01643408076246189
CMI: 0.006628327254720734
CMI: 0.010663341096499218
CMI: 0.010231621632469518
CMI: 0.009172002127631612
CMI: 0.006387084286366029
CMI: 0.01052408408987296
CMI: 0.007386043178964352
CMI: 0.011261682553921404
CMI: 0.00341068081992274
CMI: 0.004883800999764237
CMI: 0.002295899412896679
CMI: 0.0037708126867313507
CMI: 0.006834302051877128
CMI: 0.006730089899804337
CMI: 0.0027246595594461465
CMI: 0.0006166574011510773
CMI: 0.0046423744366293485
CMI: 0.011046746053095108
CMI: 0.004583467165971522
CMI: 0.005669697998885642
CMI: 0.008388088296625518
CMI: 0.014250345605013703
CMI: 0.00617495741316762
CMI: 0.009317916446412805
CMI: 0.006902821309760193
CMI: 0.004577377831586682
CMI: 0.003907273799393429
CMI: 0.00591047801146069
CMI: 0.0066021903312790675
CMI: 0.002509903625459664
CMI: 0.0018014775064681965
CMI: 0.013088530414519045
CMI: 0.00473017

actual training score: 0.17420699449918053
actual validation score: 0.47107643220662443, number of remaining columns: 261

actual training score: 0.17420780806523817
actual validation score: 0.47128558106773966, number of remaining columns: 260

actual training score: 0.17434798287370445
actual validation score: 0.47131962552222784, number of remaining columns: 259

actual training score: 0.17436549476210694
actual validation score: 0.47126401568207754, number of remaining columns: 258

actual training score: 0.17464802211326347
actual validation score: 0.47098196227583844, number of remaining columns: 257

actual training score: 0.17562074144321138
actual validation score: 0.47006151076081204, number of remaining columns: 256

actual training score: 0.1762448907741606
actual validation score: 0.4687041011118236, number of remaining columns: 255

actual training score: 0.17722696457430187
actual validation score: 0.4683174998443813, number of remaining columns: 254

actual training sco

CMI: 0.0283234203591749
CMI: 0.019318484385180162
CMI: 0.011230927814358277
CMI: 0.05020781394431609
CMI: 0.008799258655410924
CMI: 0.017481388266692655
CMI: 0.03986547721135539
CMI: 0.02863191902565232
CMI: 0.01725946538668019
CMI: 0.03124065641314644
CMI: 0.033788882439772905
CMI: 0.026654967178634636
CMI: 0.036273145410394225
CMI: 0.03747938122158612
CMI: 0.03176119969303112
CMI: 0.024628688868409132
CMI: 0.025155430928290506
CMI: 0.02372881725361281
CMI: 0.0319430019213712
CMI: 0.025724586419427126
CMI: 0.022214698821486772
CMI: 0.015051915669668015
CMI: 0.003355400956716742
CMI: 0.020607574022817016
CMI: 0.012436573800666068
CMI: 0.01827602277575413
CMI: 0.004168056947754795
CMI: 0.014207680036275816
CMI: 0.008994045983245852
CMI: 0.0043708898645373195
CMI: 0.013066216589504576
CMI: 0.01009435276934903
CMI: 0.011614923746179923
CMI: 0.007141233463432373
CMI: 6.346177115013063e-05
CMI: 0.008660124967988708
CMI: 0.016642943681400263
CMI: 0.008814298534163378
CMI: 0.00770327985129934